# RWJF open data analysis

We have access to two datasets about the projects that RWJF support:

1. Pioneers dataset: Information about grants awarded as part of the Pioneers programme, which focuses on innovations in the USA
2. Global dataset: Grants awarsed as part of the Global programme, which focuses on innovations outside the USA
3. Open dataset: With information about all their grants

1 and 2 are relatively unstructured but contain rich text, whereas 3 is well structured but doesn't have a lot of text.

We want to rapidly process these data and analyse them to understand: 

* What is RWJFs funding portfolio: what topics are they supporting? Where?
* Enrich these data with additional information from for example GRID, CrunchBase to map collaboration networks.



## Preamble

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#Additional imports
import os
import ratelim
import re
import io
import urllib
import codecs
import bs4

import numpy as np
import pandas as pd

from datetime import datetime
from nltk.corpus import stopwords

stop = stopwords.words('English')

from lda_pipeline import LdaPipeline, CleanTokenize


In [ ]:
%matplotlib inline
#NB I open a standard set of directories

#Paths

#Get the top path
top_path = os.path.dirname(os.getcwd())

#Create the path for external data
ext_data = os.path.join(top_path,'data/external')

#Raw path (for html downloads)

raw_data = os.path.join(top_path,'data/raw')

#And external data
proc_data = os.path.join(top_path,'data/processed')

fig_path = os.path.join(top_path,'reports/figures')

#Get date for saving files
today = datetime.today()

today_str = "_".join([str(x) for x in [today.day,today.month,today.year]])

## 1. Load data

### Load the Global and Pioneers data

In [ ]:
def get_project_meta(project):
    '''
    This function takes a project and returns the name and the id (if they are available, this is not always the case)
    
    '''
    
    if 'ID'  in project:
        #Split on the ID string to get the name
        name = project.split('ID')[0].strip()
        
        #Split on the ID string again to get what we want
        grant_id = re.sub('[#:]','',project.split('ID')[1].split('\n')[0].strip()).strip()

    
    else:
        #If there is no ID we split on line breaks
        name = project.strip().split('\n')[0].strip()
        grant_id = np.nan
     
    #description = project.split('\n*')[1]
    return([name,grant_id])

#def clean_up_text(project):
    


def flatten_list(my_list):
    '''
    Turns a nested list into a flat list
    
    '''
    
    flat = [x for el in my_list for x in el]
    
    return(flat)

In [ ]:
def read_rwjf_data(file):
    '''
    This function reads project lists from the RWJF and tidies it up, and returns
    a list where each element has the project name, grant id and description
    
    '''
    #Load the data
    with open(ext_data+'/'+file, 'r') as myfile:
        data=myfile.read()
    
    #Split it based on the project separator and leave out the links at the top
    projects = data.split('\n________________\n')[1:]
    
    #Extract metadata
    project_meta = [get_project_meta(x) for x in projects]
    
    #project_descriptions = [x[2] for x in project_meta]
    
    #Clean up the project info
    projects_clean = [re.sub('\* ','',re.sub('\n','',project)).lower() for project in projects]
    
    return([[x,y,z] for x,y,z in zip(
        [x[0] for x in project_meta],
        [x[1] for x in project_meta],
        projects_clean)])
    
    
    
    

In [ ]:
#Load both files
pio = read_rwjf_data('pioneer_grantees.txt')

glob = read_rwjf_data('global_grantees.txt') 

In [ ]:
rw_df = pd.DataFrame([x+['pioneers'] for x in pio]+[x+['global'] for x in glob],columns=['project',
                                                                                'code','description','source'])

rw_df.to_csv(proc_data+'/{date}_rwj_data_from_mike.csv'.format(date=today_str))

rw_df.head()

In [ ]:
descriptions = rw_df['description']
source = rw_df['source']

index_pad_length = len(str(max(descriptions.index)))

with open(proc_data + '/rwjf_pioneers_and_globals_for_mti.txt', 'w') as f:
    for i, t, s in zip(descriptions.index, descriptions.values, source.values):
        s = s[0]
        if pd.isnull(t):
            t = 'None'
        f.write('{}{}|{}\n'.format(s, str(i).rjust(index_pad_length, '0'),
                                 t.replace("\n", " ").replace("\r", " ").replace("\t", " ").encode('ascii', 'ignore')))

In [ ]:
rw_df.iloc[0]['description']

### Load the RWJF open grant data

In [ ]:
grant_data = pd.read_csv(ext_data+'/rwjf_grants.csv')

In [ ]:
len(grant_data)

In [ ]:
grant_data.head()

In [ ]:
#Unfortunately they don't have the grant ids in their open dataset! 

pd.Series(flatten_list([[y for y in x[2].split(' ') if y not in stop] for x in pio])).value_counts()[:10]


## Load the RWJF scraped data

In [ ]:
with open(ext_data+'/6_4_2018_rwjf_scraped.json','r') as infile:
    rwf = json.load(infile)
    
r_df_messy = pd.concat([pd.Series(x) for x in rwf],axis=1).T

In [ ]:
#This is what the data looks like. Note the presence of 'Nones' which we want to turn into nans and
# line breaks \n which we want to turn into spaces
#NB also the presence of apparently redundant columns. How is amount_awarded different from awarded?
r_df_messy.head()

In [ ]:
#Get the tipe of each class. They are mostly strings. We will need to convert the awarded to floats,
#Timeframe or awarded on to dates and so forth.
r_df_messy.dropna(axis=0).iloc[0,:].apply(lambda x: type(x))

In [ ]:
#Just checking the formats. Looks like the minimum value that RWJ funds is $50K
r_df_messy.apply(lambda x: x.describe())

In [ ]:
#Tidying up. Add missing values
r_df_messy = r_df_messy.applymap(lambda x: np.nan if (x==None) | (x=='') else x)

#Remove \n 
#r_df_messy = r_df_messy.applymap(lambda x: re.sub('\n',' ',x) if type(x)==str else x)

#This is what it looks like now
r_df_messy.head()

In [ ]:
def get_year(date_string):
    '''
    Extracts the year from a date string
    '''
    
    year = int(date_string.split('/')[-1]) if type(date_string)==str else np.nan
    return(year)

In [ ]:
#Here we convert the amounts awarded to numbers, the awarded dates and start dates to dates.

r_df_messy['amount_awarded_$'] = r_df_messy['amount_awarded'].apply(lambda x: 
                                                                  int(re.sub(r'[$,]','',x)) if type(x)==str else np.nan)

r_df_messy['awarded_$'] = r_df_messy['awarded'].apply(lambda x: 
                                                                  int(re.sub(r'[$,]','',x)) if type(x)==str else np.nan)

#Topics
r_df_messy['topics'] = r_df_messy['topics'].apply(lambda x: x.split('\n') if type(x)==str else np.nan)

#Years are strings
r_df_messy['year'] = r_df_messy['year'].apply(lambda x: int(x) if type(x)==str else np.nan)


In [ ]:
#Select variables
selected_variables = ['grant_number',
            'title','about','topics',
            'organization','address','website','location',
            'year','amount_awarded_$','awarded_$',]


rwj = r_df_messy[selected_variables]

In [ ]:
rwj.head()

## 2. Exploratory data analysis

Let's learn more about these data:

* What do different variables mean?
* What are the trends in terms of activity?
* What organisations are being funded?
* In what topics?



### What do the financials mean?

In [ ]:
#Amount awarded is present in almost all cases. Is it the same as awarded?
rwj[['amount_awarded_$','awarded_$']].describe()

In [ ]:
#Let's plot this
plt.scatter(rwj['amount_awarded_$'],rwj['awarded_$'],alpha=0.5,color='blue')
plt.title('Amounts awarded vs Awarded')

TODO: Determine what's happening with these two figures

### What is the situation with missing values?

In [ ]:
def missing_props(df,ct=None):
    '''
    Utility function which takes a df and returns missing values in each variable as a % of total.
    If ct!= None this is a variable to crosstab the missing data against
    
    '''
    if ct==None:
        missing = np.round(100*df.apply(lambda x: np.sum(x.isna()))/len(df),3)
        
    if ct!=None:
        missing = {var: df.groupby(ct)[var].apply(lambda x: np.mean(x.isna())) for var in df.columns}
    
    return(missing)

In [ ]:
#Plot missing values
missing_props(rwj).sort_values(ascending=False).plot.bar(color='blue',
                                                               title='Missing values (%) by variable')

In [ ]:
fig,ax = plt.subplots(nrows=3,figsize=(8,8),sharex=True,sharey=True)

missing_props(rwj,'year')['about'].plot.bar(color='blue',ax=ax[0],title='about')
missing_props(rwj,'year')['website'].plot.bar(color='blue',ax=ax[1],title='website')
missing_props(rwj,'year')['topics'].plot.bar(color='blue',ax=ax[2],title='topics')

`About` coverage is not so good. `Website` coverage improves in the last 10 years. 
`topics` coverage has a more or less constant missing rate

**Implications** for now: If we wanted to do an analysis using website data we would need to focus in 2010s or explore sources of bias before 2010.

### Some trends

In [ ]:
# Number of projects and amounts awarded (including our two measures)

#Create a table with totals raised by year
raised = pd.concat([rwj['year'].value_counts(),rwj.groupby('year')[['amount_awarded_$','awarded_$']].sum()],axis=1)

raised['awarded_per_project'] = raised['awarded_$']/raised['year']

#Plot
fig,ax = plt.subplots(figsize=(8,10),
                      #sharex=True,
                      nrows=4)

raised['year'].plot(ax=ax[0],title='Project count',color='blue')
raised['amount_awarded_$'].plot(ax=ax[1],title='Amount awarded ($)',color='blue')
raised['awarded_$'].plot(ax=ax[2],title='Awarded ($)',color='blue')
raised['awarded_per_project'].plot.bar(ax=ax[3],title='Awarded per project $',color='blue')

[axis.get_xaxis().set_visible(False) for axis in [ax[0],ax[1],ax[2]]]



In [ ]:
#Very strong correlation between years
raised.corr(method='pearson')

No clear trend in activity. `amount_awarded` and `awarded` have a very strong correlation. They are picking up the same variable

### Top organisations and domains. This will be important for the scraping analysis

In [ ]:
#What are the most 'popular' organisations in the data in terms of number of projects and 
# amounts raised?

#Create a df with the counts / totals raised
org_activity = pd.concat([rwj['organization'].value_counts(),
                          rwj.groupby('organization')['awarded_$'].sum()],axis=1)

#Sort values by number of projects
top_organisations = org_activity.sort_values('organization')[-30:]


#plot
fig,ax = plt.subplots(ncols=2,figsize=(8,8),sharey=True)

top_organisations['organization'].plot.barh(ax=ax[0],color='blue')
top_organisations['awarded_$'].plot.barh(ax=ax[1],color='blue')

Lots of schools of public health and universities. We can look for similar initiatives / groups in other datasets? 

In [ ]:
#These are the top websites
rwj['website'].value_counts()[:20]

In [ ]:
#Addresses will need to be geocoded. It would be trivial to extract postcodes with a list of US states.
rwj['address'].value_counts()[:10]

In [ ]:
#Should check how many of these are not in the US, and if they are not in the US, where are they

### Top topics

What are the top topics in the data? What are the top words, based on the 'abouts'? What
are the funding trends for different areas?

In [ ]:
def flatten_list(list):
    '''
    Flatten a list with nested elements
    '''
    
    flat = [x for el in list for x in el]
    return(flat)
    

In [ ]:
#Here are the topics. These are interesting labels to consider when looking at health outcomes
topic_counts = pd.Series(flatten_list(rwj['topics'].dropna())).value_counts()

real_topics = topic_counts[topic_counts>5].index

In [ ]:
#We would want to look at trends: Number of appearances of topics per year
#This list comprehension

#Flattens a list of topics for a year (i.e. gives the count of topics for that year) and puts in a dataframe
year_freq = pd.DataFrame({y:pd.Series(flatten_list(list(rwj.loc[rwj['year']==y,'topics'].dropna()))).value_counts() for y in 
                          range(2007,2018)}).fillna(0).loc[real_topics].T

#Calculate rolling means
year_freq_rolling = year_freq.rolling(window=3).mean()

#Plot
fig,ax = plt.subplots(figsize=(12,5))

year_freq_rolling.plot(ax=ax,linewidth=3,cmap='tab20',title='Number of projects per year (3-year Rolling averages)')

ax.legend(bbox_to_anchor=(1,1))


It seems that system-level determinants of health (*social* and *built environment*) and *early childhood development* have become more important. *Childhood obesity* has lost importance

In [ ]:
# Now I want to create similar graphs but for totals funded. How do we do this?

def filter_on_element(df,filter_variable,filter_value):
    '''
    This function takes a df where one variable filter_var is a list where every list is a nested element 
    and returns a boolean telling us if the list hasa filter value or not
    We need to run this in a df with no missing values.
    
    '''
    
    #Simple
    
    #We can't filter on missing values so we ignore them
    
    df_with_values = df.dropna(axis=0,subset=[filter_variable])
    
    #Subset the df with values on the filter variable
    filtered_df = df_with_values.loc[[filter_value in items for items in df_with_values[filter_variable]],:]

    #Return Boolean.
    return(filtered_df)
    
    

In [ ]:
#This is a complicated list comprehension. Let's see how it works.
#For each year, it loops over the topics and creates a one-element named Series with the totals raised
#by projects with the topic. It concatenates it over the year, and over all years.

awarded_year_topic = pd.concat([pd.concat([pd.Series(
    filter_on_element(rwj.loc[rwj.year==y],'topics',topic).groupby('year')['awarded_$'].sum(),
    name=topic)for topic in real_topics],axis=1) for y in np.arange(2007,2018)])


#Calculate rolling means
year_awarded_rolling = awarded_year_topic.rolling(window=3).mean()

#Plot
fig,ax = plt.subplots(figsize=(12,5))

year_awarded_rolling.plot(ax=ax,linewidth=3,cmap='tab20',title='Awarded to project with topic (3-year Rolling averages)')

ax.legend(bbox_to_anchor=(1,1))



In [ ]:
#Correlations between topics
#Focus on df that has topics
rwj_with_topics = rwj.dropna(axis=0,subset=['topics'])

#Create a topic boolean
topic_bool = pd.concat([pd.Series(
    [1 if top in x else 0 for x in rwj_with_topics['topics']],name=top) for top in real_topics],axis=1)

In [ ]:
#Create a similarity metric and visualise
#We need to import the metrics
from sklearn.metrics.pairwise import pairwise_distances

#Similarities are the opposite of distances
topic_distances = 1- pairwise_distances(topic_bool.T,metric='jaccard')

np.fill_diagonal(topic_distances,np.nan)

In [ ]:
fig,ax = plt.subplots(figsize=(8,5))

im = ax.imshow(topic_distances,aspect='auto',cmap='seismic')
ax.set_xticks(np.arange(0,15))
ax.set_xticklabels(topic_bool.T.index,rotation=45,ha='right')

ax.set_yticks(np.arange(0,15))
ax.set_yticklabels(topic_bool.T.index)

fig.colorbar(im)

ax.set_title('Similarity between topics based on co-occurrence in projects')

Frequent topics are more similar. Hard to discern is this is an artifact of the way the projects are labelled or of their actual similarity

### Top words

And now we focus on words. This will be important because it will determine if we can use NLP on topic descriptions

In [ ]:
#Remove the abouts
rwj_has_ab = rwj.dropna(axis=0,subset=['about'])


In [ ]:
#Create a tokenised about
#Too many chained methods here?

#NB we are using 'threshold' to extract phrases from the corpus

rwj_has_ab['about_tokenised'] = CleanTokenize(rwj_has_ab['about']).clean().bigram(threshold=100).tokenised

rwj_has_ab['token_length'] = [len(x) for x in rwj_has_ab['about_tokenised']]

In [ ]:
#What are the top words
token_freqs = pd.Series(flatten_list(rwj_has_ab['about_tokenised']),name='freq').value_counts()

#The top words are not very informative
token_freqs[:20]

In [ ]:
print(token_freqs.describe())

print('\n')

print(np.sum(token_freqs>10))

There are 55732 tokens in the corpus. The top word is health (unsurprisingly!). The median word appears 2 times
and there are 7600 tokens that appear more than 10 times. We might be able to use this in a topic modelling exercise or classification of projects into health areas.

In [ ]:
plt.plot(rwj_has_ab.groupby('year')['token_length'].mean())

## Exploratory topic modelling

Here we perform an exploratory topic modelling of the RWJF data. We simply want to map the activities being undertaken.

We use a script we have created with a standard LDA pipeline based on Gensim.

We will then explore the results using pyLDA viz


In [ ]:
print('running 1')

test_norm = LdaPipeline(rwj_has_ab['about_tokenised']).filter(3).process().fit_lda(
    num_topics=50,passes=20,iterations=130)

print('running 2')

test_tf = LdaPipeline(rwj_has_ab['about_tokenised']).filter(3).process().tfidf().fit_lda(
    num_topics=50,passes=20,iterations=130)

In [ ]:
test_norm.lda_topics

The topics are a bit hit and miss. We seem to be picking up health issues rather than health innovations. Perhaps we need more text?

#### Explore the data using PyLDAvis?

Unfortunately pyLDAvis doesn't seem to play nice with JupyterLabs yet. We have to display the visualisations in a different window


In [ ]:
#We will display it separately
import pyLDAvis.gensim
#pyLDAvis.enable_notebook()

#ldavis_norm = pyLDAvis.gensim.prepare(test_norm.lda_model,test_norm.corpus,test_norm.dictionary)
#pyLDAvis.save_html(ldavis_norm,fig_path+'/{date}_test_viz.html'.format(date=today_str))

#ldavis_tfidf = pyLDAvis.gensim.prepare(test_tf.lda_model,test_tf.corpus,test_tf.dictionary)
#pyLDAvis.save_html(ldavis_tfidf,fig_path+'/{date}_test_viz_tfidf.html'.format(date=today_str))

### Word embedding and document embedding analysis

Another option is to cluster the documents using Doc2vec, which will represent each project in a vector space based on the semantic similarity between its words. We can then cluster these documents and extract their top words in order to identify what they are about.

Activities: 

* Train the doc2vec model on the data.
* Cluster the docs
* Benchmark the clusters
* Label the docs

In [ ]:
from gensim.models.doc2vec import TaggedDocument
from gensim import models

#This creates a list where every element is a tag (doc title) and a list of words.
#We train the model on that
sents = [TaggedDocument(tags=[x],words=y) for x,y in zip(rwj_has_ab['title'],rwj_has_ab['about_tokenised'])]

#Train model (NB I havent tuned the d2v)
d2v = models.Doc2Vec(sents)

#Create document vector matrix
document_vectors = np.array([d2v.docvecs[t] for t in rwj_has_ab['title']])


### Clustering analysis

We want to cluster the documents.

We will build a pipeline to do this. How is it going to work?

* Initialise the class with the records (nrows = records, columns=features for clustering)
* Input the parameters for grid-search (eg number of clusters, other variables)
* Estimate silouhette scores to compare model performance across algorithms and parameters
* Return models
* Select models
* Re-run models for robust allocation of observations to clusters via community detection
* Obtain and name clusters (we will create a function that does this based on the salient terms in the cluster group)
* We can then explore the occurrence of technology areas across these clusters.
* There is no reason why we couldn't cluster the documents using other vector representations (eg topics via
LDA etc.)




In [ ]:
#Imports

from itertools import product

from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, MeanShift, SpectralClustering
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, silhouette_samples

from yellowbrick.cluster.elbow import KElbowVisualizer
from yellowbrick.cluster.silhouette import SilhouetteVisualizer
from yellowbrick.text import TSNEVisualizer

In [ ]:
class ClusterDecider():
    '''
    This class is initialised with a df or array where: 
    
        -Rows are the elements we want to cluster
        -Columns are the features we want to use for the clustering (might have done dimensionality reduction on them)
    
    Methods:
    
        
        -grid_search method taking a list of clustering algorithms and parameters to do grid search over
        [need to remember to import clustering algorithms]. This involves creating a cartesian product
        of algorithms, fitting the models and predicting the labels.
    
        TODO -evaluation method estimates metrics of clustering performance based on silouhette score. If
        we use a word_similarity metric then we will run a custom-made algorithm that estimates the 
        average distances between salient words in each cluster compared to those in other clusters,
        
        -Return a dict with all the models we fit, their parameters and scores.
        
        We will evaluate these manually, and through Yellowbrick.
    
    '''
    
    def __init__(self,records):
        '''
        Initialise an instance of the object.
        
        '''
        
        self.records = records
    
    def grid_search(self,cluster_list):
        '''
        Loops through the cluster list and creates a grid search based on the parameters
        The parameters are in the second element of each cluster, as a tuple 
        where the first value is the name of the parameter and the rest are 
        
        
        '''
        
        #Read the records
        records = self.records
        
        #Store clustering results in a dict
        self.clustering = {}
        
        #We loop over each cluster and create combinations of parameters
        for clust in cluster_list:
            cl_algo = clust[0]
            
            #We get the cluster name in a hacky way via regex
            cluster_name = re.sub('_','',str(clust[0]).split('.')[2])
            
            print('running '+cluster_name)
            
            #This extracts the dictionary of parameters from the clust list
            #and creates a cartesian product (all possible combinations of values)
            par_comb = list(product(*clust[1].values()))
            
            #And this turns the cartesian product into a list of dicts with named parameters
            par_list = [{par:val for par,val in zip(clust[1].keys(),par_vals)} for par_vals in par_comb]
            
            #Now we can loop over this list to to run each cluster:
            
            for parameters in par_list:
                #Initialise the cluster and set parameters
                cl = cl_algo().set_params(**parameters)
                
                #Fit the cluster
                cl.fit(records)
                
                #We also want to estimate the silouhette scores etc.
                sil_score = silhouette_score(records,cl.labels_)
                
                #Store results
                self.clustering['_'.join([cluster_name]+[k+':'+str(v) for k,v in parameters.items()])]=[
                    cl,
                    cl.labels_,
                    sil_score]
            
                #We also want to estimate the silouhette scores etc.   
                
        return(self)
                
    def cluster_scores(self):
        '''
        Takes the scores from the clustering algorithms and plots them
        
        '''
        
        #Extract scores
        scores = pd.DataFrame([{'name':k,'score':v[-1]} for k,v in self.clustering.items()]).sort_values('score')   
        
        fig,ax = plt.subplots()

        scores['score'].plot.bar(ax=ax,color='blue')

        ax.set_xticklabels(scores['name'])
        ax.set_title('Silouhette scores')
        
        return(ax)
        
        
        

In [ ]:
def visual_validation(cluster_labels,sort_results=True,print_n='all',subset=False):
    '''
    This function takes a list of cluster labels and returns some information about their content:
    Number of observations, titles, distinctive words...
    
    We can ask it to sort the data (start with the biggest clusters) and limit the numbers
    that are printed (if we have lots). We can also ask it to print a subset of the clusters
    
    '''
    
    rwj_clust = pd.concat([rwj_has_ab[['title','about_tokenised']]])
    rwj_clust['cluster'] = cluster_labels

    if sort_results == True:
    #If we want to present the results labelled
        cluster_labels = pd.Series(cluster_labels).value_counts().index
    
    else:
        cluster_labels = sorted(list(set(cluster_labels)))
    
    
    if subset!=False:
        cluster_labels = [x for x in cluster_labels if x in subset]
    
    if print_n!='all':
        '''
        Select how many to print
        
        '''
        
        cluster_labels = cluster_labels[:print_n]
        
    
    for x in cluster_labels:


        rel = rwj_clust.loc[rwj_clust['cluster']==x]

        print(x)
        print(len(rel))

        print(rel.head())

        print('\n')

        el_freq = pd.Series(flatten_list(rel['about_tokenised']),name=x).value_counts()[:50]

        norm_freq = pd.concat([el_freq,token_freqs],axis=1)
        norm_freq['norm'] = norm_freq[x]/norm_freq['freq']

        print(norm_freq.sort_values('norm',ascending=False)[:10])    

        print('\n')

In [ ]:
cd = ClusterDecider(document_vectors)

clusts= [
    #[SpectralClustering,{'n_clusters':np.arange(10,20,2)}],
    [KMeans,{'n_clusters':np.arange(15,40,2)}],
    #[DBSCAN,{}],
    #[MeanShift,{'cluster_all':[True,False]}],
    #[AffinityPropagation,{}]
         ]

cd.grid_search(cluster_list=clusts)

cd.cluster_scores()

In [ ]:
sel_labs = cd.clustering['kmeans_n_clusters:15'][1]

visual_validation(sel_labs,print_n=10)

### Additional processing

Currently, the clustering algorithm is picking up programmes with repeated titles.

We want to remove those. How do we do it?

Idea: cluster documents on their titles using doc2vec and then identify which clusters are
most homogeneous (eg average levehnstein distances between all components is lower. 
We then allocate all documents in 'programme clusters' in there, and redo the clustering

In [ ]:
#We tokenise titles
titles_tokenised = CleanTokenize(rwj_has_ab['title']).clean().bigram(threshold=100).tokenised

#We train the model on that
sents_dedupe = [TaggedDocument(tags=[x],words=y) for x,y in zip(rwj_has_ab['title'],titles_tokenised)]

#Train model (NB I havent tuned the d2v)
d2v_title = models.Doc2Vec(sents_dedupe)

#Create document vector matrix
document_vectors_titles = np.array([d2v_title.docvecs[t] for t in rwj_has_ab['title']])

In [ ]:
#Run the clustering as before
cd_titles = ClusterDecider(document_vectors_titles)

clusts= [
    #[SpectralClustering,{'n_clusters':np.arange(10,20,2)}],
    [KMeans,{'n_clusters':np.arange(30,80,5)}],
    #[DBSCAN,{}],
    #[MeanShift,{'cluster_all':[True,False]}],
    #[AffinityPropagation,{}]
         ]

cd_titles.grid_search(cluster_list=clusts)

cd_titles.cluster_scores()

We want to allocate 'repeated' clusters (with the same title) to the same cluster and normalise.

Otherwise any cluster analysis or topic modelling we do is going to keep picking up those repeated clusters

We identify what these clusters are with the mean/variance distance between observations and the cluster centroid (clusters with large/variant distances will be more dispersed than those with small/invariant distances, and less likely to contain repeated programmes

In [ ]:
from sklearn.metrics import pairwise


In [ ]:
test_cluster = cd_titles.clustering['kmeans_n_clusters:60'][0]

#For each cluster, we will calculate distance mean and variance between all observations and the centre.

container = []

for clust in list(set(test_cluster.labels_)):
    #print(clust)
    # Find the document vectors in the cluster
    
    #Subset the document vectors to get those in the cluster
    vectors = np.array(document_vectors_titles)[[val==clust for val in test_cluster.labels_],:]
    
    size = len(vectors)
    
    #print(vectors.shape)
    
    #Extract the centroid
    centroid = test_cluster.cluster_centers_[x]
    
    #print(centroid)
    
    #Calculate all distances
    distances = pairwise.cosine_distances(vectors,centroid.reshape(1,-1))
    
    #print(distances.shape)
    
    #Calculate distance mean and variance
    median = np.median(distances)
    
    variance = np.round(np.var(distances),3)
    
    container.append([clust,size,median,variance])

In [ ]:
zero_var = [x[0] for x in container if x[3]==0]
non_zero_var = [x[0] for x in container if x[0] not in zero_var]

#non_zero_var = [x[0] for x in container if x[0] not in zero_var]

In [ ]:
visual_validation(cd_titles.clustering['kmeans_n_clusters:60'][1],sort_results=False,
                  subset=zero_var)

Ok - this seems to be picking up the 'duplicate' programmes. Maybe we can wrap them up together and redo the 
cluster analysis